In [41]:
import requests
import warnings
import os

import pandas as pd

from pymisp import PyMISP, MISPObject, MISPAttribute
from bs4 import BeautifulSoup
from dotenv import load_dotenv

In [42]:
load_dotenv()

misp_api_key = os.getenv("MISP_API")

In [43]:
securityBulletin = pd.DataFrame(columns=('RefernceN', 'Title', 'ArticleLink'))

# The first 4 pages
for pageN in range(0, 4):
    URL = f"https://www.dgssi.gov.ma/fr/macert/bulletins-de-securite.html/page/{pageN}"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.findAll('a', class_="title_event")

    resultsDF = pd.DataFrame(
        ([title.text.strip().split('-')[0], "".join(title.text.strip().split('-')[1:]), "https://www.dgssi.gov.ma" + title.find('a')['href']] for title in results),
        columns=('RefernceN', 'Title', 'ArticleLink')
    )

    securityBulletin = pd.concat([securityBulletin, resultsDF], ignore_index=True)

securityBulletin

,RefernceN,Title,ArticleLink
0,40710803/23,Exploitation d’une faille critique dans Micro...,https://www.dgssi.gov.ma/fr/content/4071080323...
1,40680703/23,Vulnérabilités critiques affectant le thème «...,https://www.dgssi.gov.ma/fr/content/4068070323...
2,40690703/23,Vulnérabilités dans les produits IBM,https://www.dgssi.gov.ma/fr/content/4069070323...
3,40650703/23,Vulnérabilités affectant plusieurs produits d...,https://www.dgssi.gov.ma/fr/content/4065070323...
4,40610603/23,SkullLocker Ransomware,https://www.dgssi.gov.ma/fr/content/4061060323...
5,40630303/23,Vulnérabilités affectant Zimbra Collaboration,https://www.dgssi.gov.ma/fr/content/4063030323...
6,40620303/23,Vulnérabilités affectant plusieurs produits ...,https://www.dgssi.gov.ma/fr/content/4062030323...
7,40610203 /23,Nouvelle compagne d’attaque par le malware « ...,https://www.dgssi.gov.ma/fr/content/40610203-2...
8,40600203/23,Vulnérabilité affectant le navigateur Mozill...,https://www.dgssi.gov.ma/fr/content/4060020323...
9,40590203/23,Vulnérabilités affectant le navigateur Micro...,https://www.dgssi.gov.ma/fr/content/4059020323...


In [73]:
def add_group_fromDF(MISPInstance, eventID, df):
    for i, row in df.iterrows():
        object = MISPObject("Blog", standalone=False)

        object.add_attribute("title", value=row["Title"], type="text")
        object.add_attribute("link", value=row["ArticleLink"], type="link")
        
        MISPInstance.add_object(eventID, object)

In [74]:
warnings.filterwarnings("ignore")

## Ignoring the certificate verification

misp_url = "https://localhost/"
misp_key = misp_api_key
misp_verifycert = False

myMISPInstance = PyMISP( misp_url, misp_key, misp_verifycert, "json" )

## Example
add_group_fromDF(myMISPInstance, "5", securityBulletin)